<a href="https://colab.research.google.com/github/MatiasBlanco0/Trabajo-Practico-Netflix/blob/main/Trabajo_Practico_Netflix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Librerias

In [ ]:
from google.colab import files
import pandas as pd

Importamos `files` de `google.colab` para poder subir el archivo csv.

Importamos `pandas` para poder leer y modificar el archivo csv.

# Subir Archivo CSV

In [ ]:
uploaded = files.upload()

Saving NetflixHistory.csv to NetflixHistory.csv


# Leer y filtrar el archivo CSV

In [ ]:
df = pd.read_csv("/content/NetflixHistory.csv")

Guardamos el archivo csv en una variable llamada `df`por dataframe

In [ ]:
devicesToIgnore = ['Sony PS4', 'Netflix Windows App - Cadmium Windows Mobile', 'Apple iPhone 6s', 'Apple iPhone 7 Plus (CDMA)', 'Chrome PC (Cadmium)', 'Samsung 2015 NT14M DTV Smart TV', 'iPhone 5S GSM', 'Google Chromecast V3 Streaming Stick']
for deviceToIgnore in devicesToIgnore:
  df.drop(df[df['Device Type'] == deviceToIgnore].index, inplace=True)

Correr esto para filtrar los dispositivos indicados. `inplace=True` sirve para que la funcion sobreescriba la variable en vez de devolver una copia

# Ejercicios

## A. Contabilizar el dispositivo con el que más títulos se reprodujeron.

Nosotros contamos como titulo al titulo unico, es decir, si se ve una pelicula dos veces, esto cuenta como una vez

In [ ]:
devices = {}
# Iteramos por cada dispositivo del dataframe y si no esta lo agregamos con un valor
for device in df['Device Type']:
  if device not in devices:
    # Sacamos la cuenta de valores de la serie Title donde el Device Type es igual al del device actual, lo convertimos a un diccionario y sacamos las keys, porque es lo que nos interesa (los nombres de los titulos)
    # Al final devolvemos el largo para saber cuantos titulos se vieron en ese dispositivo
    devices[device] = len(df[df['Device Type'] == device]['Title'].value_counts().to_dict().keys())

# Utilizamos la funcion sorted para ordenar en base a los valores, accediendolos mediante devices.items() y en el callback de key devolviendo el valor. Ademas de decir que los ordene de mayor a menor
# Esto de devuelve una lista de tuplas, por eso despues los convertimos a un diccionario
devices = dict(sorted(devices.items(), key=lambda item: item[1], reverse=True))
devices

{'Apple Apple TV 4 Apple TV': 1920,
 'Apple TV 3 (revised)': 828,
 'Netflix Chrome MAC (Cadmium) HTML 5': 285,
 'Safari MAC (Cadmium)': 219,
 'Apple iPhone 4S': 211,
 'Apple iPod Touch 5G': 143,
 'LG 2016 LG M16 STD RGBW UHD TV Smart TV': 76,
 'Wii Bowser': 45,
 'Apple iPad 3': 35,
 'Apple iPhone 11 Pro': 34,
 'LG 2016 MStar M2 Low DTV Smart TV': 31,
 'Apple iPad 3rd Gen (Wi-Fi Only) iPad': 10,
 'LG 2017 LG M16p Standard UHD TV Smart TV': 2,
 'Apple iPhone 13 Mini': 1,
 'Android DefaultWidevineL3Tablet Android Tablet': 1}

El dispositivo en el que mas titulos se reprodujeron fue: Apple Apple TV 4 Apple TV, con 1920 titulos

## B. Encontrar la película con más ocurrencias y la película con mayor duración total.


## C. Generar un TOP “N” de series vistas y graficar.
## 1.   Por tiempo visto
## 2.   Por ocurrencias

## D. Visualizar una semana de reproducciones genérica, en función a las reproducciones por día y por horario de todos los años.


## E. Contabilizar la cantidad de títulos vistos por año, graficar su variación a través de los años.


## F. Total de días de todos los títulos reproducidos.
